In [ ]:
%pip install -r requirements.txt

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import wbgapi as wbg

In [7]:
from pygooglenews import GoogleNews

In [8]:
def get_news_dataframe(query, max_articles=100):
    # Initialize GoogleNews API
    gn = GoogleNews(lang='en', country='US')

    # Search for news articles
    search_results = gn.search(query)
    
    # Extract articles
    articles = []
    for entry in search_results['entries'][:max_articles]:
        title = entry.get("title", "N/A")
        link = entry.get("link", "N/A")
        published = entry.get("published", "N/A")
        source = entry.get("source", {}).get("title", "Unknown")
        
        # Extract topic and subtopic from categories (if available)
        topics = entry.get("tags", [])
        main_topic = topics[0]["term"] if topics else "General"
        sub_topic = topics[1]["term"] if len(topics) > 1 else "N/A"

        articles.append({
            "Title": title,
            "Link": link,
            "Published": published,
            "Source": source,
            "Topic": main_topic,
            "Subtopic": sub_topic
        })

    # Convert to DataFrame
    df = pd.DataFrame(articles)
    return df

In [9]:
query = "Apple"
news_df = get_news_dataframe(query)
news_df

,Title,Link,Published,Source,Topic,Subtopic
0,U.K. orders Apple to let it spy on users’ encr...,https://news.google.com/rss/articles/CBMiigFBV...,"Fri, 07 Feb 2025 13:32:03 GMT",The Washington Post,General,N/A
1,Apple’s Long-Awaited Overhaul of iPhone SE Nea...,https://news.google.com/rss/articles/CBMisAFBV...,"Thu, 06 Feb 2025 23:16:51 GMT",Bloomberg,General,N/A
2,Apple’s iPhone SE 4 Might Launch Way Sooner Th...,https://news.google.com/rss/articles/CBMisAFBV...,"Fri, 07 Feb 2025 01:16:57 GMT",Forbes,General,N/A
3,Apple Quickly Releases New iPhone Update But O...,https://news.google.com/rss/articles/CBMisgFBV...,"Fri, 07 Feb 2025 13:30:58 GMT",Forbes,General,N/A
4,iOS 18.3.1—What’s In Apple’s Mystery New iPhon...,https://news.google.com/rss/articles/CBMiqwFBV...,"Thu, 06 Feb 2025 17:48:49 GMT",Forbes,General,N/A
...,...,...,...,...,...,...
92,ELEGNT: Expressive and Functional Movement Des...,https://news.google.com/rss/articles/CBMihwFBV...,"Sun, 26 Jan 2025 00:21:39 GMT",Apple Machine Learning Research,General,N/A
93,"Severance Season 2, Episode 4 Review: I Can't ...",https://news.google.com/rss/articles/CBMia0FVX...,"Fri, 07 Feb 2025 08:01:00 GMT",Screen Rant,General,N/A
94,"Rumor Replay: Apple’s M5 chip is coming, iPhon...",https://news.google.com/rss/articles/CBMinwFBV...,"Thu, 06 Feb 2025 21:08:00 GMT",9to5Mac,General,N/A
95,Apple TV+ unveils a delicious first look at Fr...,https://news.google.com/rss/articles/CBMi5wFBV...,"Wed, 05 Feb 2025 14:12:15 GMT",Apple Newsroom,General,N/A


In [10]:
GDP = 'NY.GDP.MKTP.KD'       # GDP in constant 2015 $US
GDPPC = 'NY.GDP.PCAP.KD'     # GDP per capita in constant 2015 $US
CPI = 'FP.CPI.TOTL.ZG'       # Inflation rate
interest = "FR.INR.RINR"     # interest rate
unemployment = "SL.UEM.TOTL.ZS" # Unemployment rate

Table_1 = wbg.data.DataFrame([GDP, GDPPC, CPI, interest,unemployment], 'USA', time=range(2015, 2025), numericTimeKeys=True, labels=True, columns='series')
Table_1 = pd.DataFrame(Table_1)

In [11]:
Table_1.rename(columns = {'NY.GDP.MKTP.KD' : 'GDP', 'NY.GDP.PCAP.KD' : 'GDPPC', 'FP.CPI.TOTL.ZG' : 'CPI', 'BX.KLT.DINV.WD.GD.ZS' : 'FDI', 'FR.INR.RINR' : 'Interest rate', 'SL.UEM.TOTL.ZS' : 'Unemployment rate'}, inplace = True)
Table_1 = Table_1.sort_values(by = ['Time'], ascending = True)
Table_1 = Table_1.reset_index()
Table_1

,time,Time,CPI,Interest rate,GDP,GDPPC,Unemployment rate
0,2015,2015,0.118627,2.310515,1.829502e+13,57040.208214,5.280
1,2016,2016,1.261583,2.537232,1.862789e+13,57658.670883,4.869
2,2017,2017,2.130110,2.265296,1.908569e+13,58703.144078,4.355
3,2018,2018,2.442583,2.554750,1.965187e+13,60127.210276,3.896
4,2019,2019,1.812210,3.573062,2.015964e+13,61400.548154,3.669
5,2020,2020,1.233584,2.186282,1.972358e+13,59493.145982,8.055
6,2021,2021,4.697859,-1.258522,2.091785e+13,62996.289384,5.349
7,2022,2022,8.002800,NaN,2.144339e+13,64342.117938,3.650
8,2023,2023,4.116338,NaN,2.206258e+13,65875.177881,3.638


In [12]:
from datetime import datetime
end = datetime.now()
start = datetime(year=end.year-10, month=end.month, day=end.day)

In [15]:
stock_data = yf.download(tickers='AAPL',start=start,end=end)
stock_data = stock_data.reset_index()
stock_data.info()

[*********************100%***********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2515 entries, 0 to 2514
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   (Date, )        2515 non-null   datetime64[ns]
 1   (Close, AAPL)   2515 non-null   float64       
 2   (High, AAPL)    2515 non-null   float64       
 3   (Low, AAPL)     2515 non-null   float64       
 4   (Open, AAPL)    2515 non-null   float64       
 5   (Volume, AAPL)  2515 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 118.0 KB


In [16]:
import ta